In [1]:
import glob
import os
import librosa
import numpy as np
import tensorflow as tf

import h5py

def bringdata(filename):
    hdf5Path = filename
    dataset = h5py.File(hdf5Path, 'r')
    data = dataset['audio']
    labels = dataset['labels']
    print 'data brought', filename
    print data.shape, labels.shape
    return data, labels

traindata, trainlabel = bringdata('train')
testdata, testlabel = bringdata('test')

data brought train
(48840, 60, 41, 2) (48840, 10)
data brought test
(5218, 60, 41, 2) (5218, 10)


In [2]:
import matplotlib.pyplot as plt

# matplotlib inline
plt.style.use('ggplot')
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['legend.fontsize'] = 11
plt.rcParams['figure.titlesize'] = 13

In [3]:
import tensorflow as tf

frames = 41
bands = 60
num_channels = 2

#feature_size = 2460 #60x41
num_labels = 10

X = tf.placeholder(tf.float32, shape=[None,bands,frames,num_channels])
Y = tf.placeholder(tf.float32, shape=[None,num_labels])

keep_prob1 = tf.placeholder(tf.float32)  # use for dropout

In [4]:
tf.reset_default_graph()

In [5]:
# conv1
with tf.variable_scope('conv1') as scope:
    weights = tf.get_variable('weights', shape=[5, 5, 2, 256], dtype=tf.float32,
                              initializer=tf.contrib.layers.xavier_initializer(seed=0))

    biases = tf.get_variable('biases', shape=[256], dtype=tf.float32,
                             initializer=tf.constant_initializer(0.1))

    conv1 = tf.nn.relu(tf.nn.conv2d(X, weights, strides=[1, 1, 1, 1], padding='SAME') + biases)

# pool1
with tf.variable_scope('pool1') as scope:
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                           padding='VALID', name='pool1')
# conv2
with tf.variable_scope('conv2') as scope:
    weights = tf.get_variable('weights', shape=[3, 3, 256, 384], dtype=tf.float32,
                              initializer=tf.contrib.layers.xavier_initializer(seed=0))

    biases = tf.get_variable('biases', shape=[384], dtype=tf.float32,
                             initializer=tf.constant_initializer(0.1))
    
    conv2 = tf.nn.relu(tf.nn.conv2d(pool1, weights, strides=[1, 1, 1, 1], padding='SAME') + biases)
    
# pool2
with tf.variable_scope('pool2') as scope:
    pool2 = tf.nn.max_pool(conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                           padding='VALID', name='pool5')
pool2_flatten = tf.contrib.layers.flatten(pool2)  # flatten pool3 to shape of (128,16*16*32)

# fully connected layer 1 and dropout
fc1 = tf.contrib.layers.fully_connected(pool2_flatten, 256, activation_fn=tf.nn.relu)
fc1_drop = tf.nn.dropout(fc1, keep_prob1)

# output
fc3 = tf.contrib.layers.fully_connected(fc1_drop, num_labels, activation_fn=None)


ValueError: Tensor("Placeholder:0", shape=(?, 60, 41, 2), dtype=float32) must be from the same graph as Tensor("conv1/weights:0", shape=(5, 5, 2, 256), dtype=float32_ref).

In [27]:
# loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
train_optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

# accuracy
correct_prediction = tf.equal(tf.argmax(fc3,1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

In [28]:
batch_size = 8
iterations = 100

with tf.Session() as session:
    tf.global_variables_initializer().run()

    # Training
    for itr in range(iterations):

        c_sum = 0

        for i in range(48840/batch_size):
            offset = (i * batch_size) % (trainlabel.shape[0] - batch_size)
            batch_x = traindata[offset:(offset + batch_size), :, :, :]
            batch_y = trainlabel[offset:(offset + batch_size), :]

            _, c = session.run([train_optimizer, cost], feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.5})
            c_sum += c / (48840/batch_size)

        cost_history = np.append(cost_history, c_sum)

        print itr, '/', iterations, 'done', 'cost:', c_sum

    fig = plt.figure(figsize=(15, 10))
    plt.plot(cost_history=True)
    plt.axis([0, iterations, 0, np.max(cost_history)])
    plt.show()

InvalidArgumentError: Input to reshape is a tensor with 84480 values, but the requested shape requires a multiple of 3136
	 [[Node: Reshape_18 = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/gpu:0"](MaxPool_7, Reshape_18/shape)]]

Caused by op u'Reshape_18', defined at:
  File "/home/jihyo/anaconda3/envs/gpuenv/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/jihyo/anaconda3/envs/gpuenv/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/jihyo/anaconda3/envs/gpuenv/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/jihyo/anaconda3/envs/gpuenv/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/jihyo/anaconda3/envs/gpuenv/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 499, in start
    self.io_loop.start()
  Fil

In [ ]:
def conv_layer(X, filter_height, filter_width, num_filters, name, stride = 1, padding = 'SAME'):
    input_channels = int(X.get_shape()[-1])
    with tf.variable_scope(name) as scope:
        weights = tf.get_variable('weights', shape=[filter_height, filter_width, input_channels, num_filters],
                                  dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=0))
        biases = tf.get_variable('biases', shape=[num_filters], dtype=tf.float32,
                                 initializer=tf.constant_initializer(0.1))
        conv = tf.nn.relu(tf.nn.conv2d(X, weights, strides=[1, stride, stride, 1], padding= padding) + biases)
        return conv

def max_pool(X, name, filter_height=3, filter_width=3, stride = 2, padding = 'SAME'):
    with tf.variable_scope(name) as scope:
        pool = tf.nn.max_pool(X, ksize=[1, filter_height, filter_width, 1], strides=[1, stride, stride, 1], padding=padding, name=name)
        return pool

def fcl(X, output_size, dropout_ratio, activation_fn= tf.nn.relu):
    fc1 = tf.contrib.layers.fully_connected(X, output_size, activation_fn = activation_fn)
    fcl_drop = tf.nn.dropout(fc1, dropout_ratio)
    return fcl_drop


def inception_module(X, conv_1_size, conv_3_reduce_size, conv_3_size, conv_5_reduce_size, conv_5_size, pool_proj_size, name = 'inception'):
    with tf.variable_scope(name) as scope:
        conv_1 = conv_layer(X, filter_height=1, filter_width=1, num_filters=conv_1_size, name='{}_1x1'.format(name))
        conv_3_reduce = conv_layer(X, filter_height=1, filter_width=1, num_filters=conv_3_reduce_size, name='{}_3x3_reduce'.format(name))
        conv_3 = conv_layer(conv_3_reduce, filter_height=3, filter_width=3, num_filters=conv_3_size, name='{}_3x3'.format(name))
        conv_5_reduce = conv_layer(X, filter_height=1, filter_width=1, num_filters=conv_5_reduce_size, name='{}_5x5_reduce'.format(name))
        conv_5 = conv_layer(conv_5_reduce, filter_height=5, filter_width=5, num_filters=conv_5_size, name='{}_5x5'.format(name))
        pool = max_pool(X, stride=1, padding='SAME', name='{}_pool'.format(name))
        pool_proj = conv_layer(pool, filter_height=1, filter_width=1, num_filters=pool_proj_size, name='{}_pool_proj'.format(name))
        return tf.concat([conv_1,conv_3,conv_5,pool_proj], axis=3, name='{}_concat'.format(name))
        

In [11]:
frames = 41
bands = 60
num_channels = 2

#feature_size = 2460 #60x41
num_labels = 10

#create placeholders
X = tf.placeholder(tf.float32, shape=[None,bands,frames,num_channels])
Y = tf.placeholder(tf.float32, shape=[None,num_labels])


In [12]:
tf.reset_default_graph() 

In [9]:
# Model
conv1 = conv_layer(X, filter_height=7, filter_width=7, num_filters=64, stride=2, name='conv1')
pool1 = max_pool(conv1, name='pool1')
conv2 = conv_layer(pool1, filter_height=3, filter_width=3, num_filters=192, stride=1, name='conv2')
pool2 = max_pool(conv2, name='pool2')
inception3a = inception_module(pool2, 64, 96, 128, 16, 32, 32, name='inception3a')
inception3b = inception_module(inception3a, 128, 128, 192, 32, 96, 64, name='inception3b')
pool3 = max_pool(inception3a, name='pool3')
inception4a = inception_module(pool3, 192, 96, 208, 16, 48, 64, name='inception4a')
inception4b = inception_module(inception4a, 160, 112, 224, 24, 64, 64, name='inception4b')
inception4c = inception_module(inception4b, 128, 128, 256, 24, 64, 64, name='inception4c')
inception4d = inception_module(inception4c, 112, 144, 288, 32, 64, 64, name='inception4d')
inception4e = inception_module(inception4d, 256, 160, 320, 32, 128, 128, name='inception4e')
pool4 = max_pool(inception4e, name='pool4')
inception5a = inception_module(pool4, 256, 160, 320, 32, 128, 128, name='inception5a')
inception5b = inception_module(inception5a, 384, 192, 384, 48, 128, 128, name='inception5b')
avg_pool = tf.nn.avg_pool(inception5b, ksize=[1,7,7,1], strides=[1,1,1,1], padding='SAME', name='avg_pool')
flatten = tf.contrib.layers.flatten(avg_pool)
fcl1 = fcl(flatten, 1024, p)
fcl2 = fcl(fcl1, 1000, 1.0)
fcl3 = fcl(fcl2, classNum, 1.0, activation_fn=tf.nn.softmax)
GoogLeNet = fcl3

ValueError: Tensor("Placeholder:0", shape=(48840, 60, 41, 2), dtype=float32) must be from the same graph as Tensor("conv1/weights:0", shape=(7, 7, 2, 64), dtype=float32_ref).

In [ ]:
# loss & train
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = GoogLeNet, labels = Y))
train = tf.train.AdamOptimizer(0.0001).minimize(loss)

# accuracy
correct_prediction = tf.equal(tf.argmax(GoogLeNet,1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

In [ ]:
batch_size = 8
iterations = 100

with tf.Session() as session:
    tf.initialize_all_variables().run()

    # Training
    for itr in range(iterations):

        c_sum = 0

        for i in range(48840/batch_size):
            offset = (i * batch_size) % (trainlabel.shape[0] - batch_size)
            batch_x = traindata[offset:(offset + batch_size), :, :, :]
            batch_y = trainlabel[offset:(offset + batch_size), :]

            _, c = session.run([train_optimizer, cost], feed_dict={X: batch_x, Y: batch_y, keep_prob1: 0.5, keep_prob2: 1.0})
            c_sum += c / (48840/batch_size)

        cost_history = np.append(cost_history, c_sum)

        print itr, '/', iterations, 'done', 'cost:', c_sum

    fig = plt.figure(figsize=(15, 10))
    plt.plot(cost_history=True)
    plt.axis([0, iterations, 0, np.max(cost_history)])
    plt.show()

    # Test
    acc = session.run(accuracy, feed_dict={X: testdata, Y: testlabel, keep_prob1: 1.0, keep_prob2: 1.0})

    print 'Test accuracy:', acc